In [ ]:
# Importing required libraries
import os
import zipfile
import numpy as np
import cv2
from google.colab import files
from sklearn.cluster import KMeans
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import time

# Upload the zip file
uploaded = files.upload()

# Extract the zip file
for filename in uploaded.keys():
    zip_file = zipfile.ZipFile(filename, 'r')
    zip_file.extractall('/content/extracted/')
    zip_file.close()

# Paths to the images
top_two_players_path = '/content/extracted/two_players_top/'
bot_two_players_path = '/content/extracted/two_players_bot/'
court_image_path = '/content/extracted/court_image/output.jpg'  # Adjusted path for output image

# Output path
output_path = '/content/output/'

# Create output folders for players
def create_output_folders():
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    for i in range(4):
        os.makedirs(os.path.join(output_path, f'player_{i+1}'))

# Load images and preprocess them for ResNet50
def load_images(image_paths):
    images = []
    for img_path in image_paths:
        img = image.load_img(img_path, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        images.append(img_data)
    return np.vstack(images)

# Extract features using ResNet50
def extract_features(images):
    model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    features = model.predict(images)
    return features

# Clustering using KMeans to segregate players
def cluster_images(features, n_clusters=4):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(features) 
    return kmeans.labels_

# Save clustered images into respective folders
def save_clustered_images(image_paths, labels):
    for img_path, label in zip(image_paths, labels):
        img_name = os.path.basename(img_path)
        dest_folder = os.path.join(output_path, f'player_{label+1}')
        cv2.imwrite(os.path.join(dest_folder, img_name), cv2.imread(img_path))

# Get image paths
def get_image_paths(folder):
    return [os.path.join(folder, img) for img in os.listdir(folder) if img.endswith(('.png', '.jpg', '.jpeg'))]

# Segment court image if needed
def segment_court_image():
    # Debugging: Check if the file exists
    if not os.path.exists(court_image_path):
        print(f"Error: Court image not found at {court_image_path}")
        return None  # Return if the file does not exist

    court_img = cv2.imread(court_image_path)

    # Debugging: Check if the image is loaded correctly
    if court_img is None:
        print(f"Error: Unable to load court image from {court_image_path}")
        return None  # Return if the image could not be loaded

    hsv = cv2.cvtColor(court_img, cv2.COLOR_BGR2HSV)

    # Define the range for court color (green in this case)
    lower_green = np.array([35, 40, 40])
    upper_green = np.array([85, 255, 255])

    # Create a mask for the court area
    mask = cv2.inRange(hsv, lower_green, upper_green)

    # Save the segmented court image (optional, for verification)
    cv2.imwrite(os.path.join(output_path, 'segmented_court.jpg'), mask)
    return mask

# Main function to process images and cluster them
def main():
    start_time = time.time()  # Start the timer

    # Create output folders for four players
    create_output_folders()

    # Load images from top and bottom player folders
    top_image_paths = get_image_paths(top_two_players_path)
    bot_image_paths = get_image_paths(bot_two_players_path)

    all_image_paths = top_image_paths + bot_image_paths
    images = load_images(all_image_paths)

    # Extract features using ResNet50
    features = extract_features(images)

    # Cluster images based on features
    labels = cluster_images(features)

    # Save the clustered images into output folders
    save_clustered_images(all_image_paths, labels)

    # Calculate execution time
    execution_time = time.time() - start_time
    print(f"Execution Time: {execution_time:.2f} seconds")

# Run the main function
main()

# Download the output folder as a zip
!zip -r /content/output.zip /content/output/
files.download('/content/output.zip')
